# Efficacité des tests de dépistage du COVID-19

On estime aujourd'hui qu'entre 3% et 6% de la population française est infectée.

Par ailleurs, la HAS a publié le [cahier des charges](https://www.has-sante.fr/upload/docs/application/pdf/2020-04/cahier_des_charges_test_serologique_covid19.pdf) des tests:

- sensibilité clinique (capacité à détecter les malades) entre 90% et 95%
- spécificité clinique (capacité à ne pas générer de faux positifs) à 98%

La figure ci-dessous permet d'illustrer l'efficacité des tests en fonction de ces données:

- en rouge, les risques d'être effectivement malade suite à un test positif (à gauche) ou à un test négatif (à droite)
- en vert les chance d'être sain

Sous la figure, différents élements de contrôle vous permettent de faire varier les caractéristiques du test et de constater leur influence sur son efficacité.

In [1]:
import numpy as np
from bqplot import OrdinalScale, LinearScale, Bars, Axis, Figure, Lines
from ipywidgets import FloatSlider, HBox, VBox, FloatText
from IPython.display import display

In [2]:
sensibility_low = 0.9
sensibility_high = 0.95
specificity_low = 0.01
specificity_high = 0.03
contamination_low = 0.03
contamination_high = 0.06

In [3]:
def positive_proba(contamination, sensibility, specificity):
    return sensibility * contamination + specificity * (1. - contamination)

def infected_if_positive(contamination, sensibility, specificity):
    return sensibility * contamination / positive_proba(contamination, sensibility, specificity)

def infected_if_negative(contamination, sensibility, specificity):
    return (1. - sensibility) * contamination / (1 - positive_proba(contamination, sensibility, specificity))

def compute_proba(contamination, sensibility, specificity):
    iip = infected_if_positive(contamination, sensibility, specificity)
    niip = 1. - iip
    iin = infected_if_negative(contamination, sensibility, specificity)
    niin = 1. - iin
    return np.array([[iip, iin], [niip, niin]])

In [9]:
x_data = ['Test Positif', 'Test Negatif']
x_ord = OrdinalScale()
y_data = compute_proba(contamination_low, sensibility_low, specificity_high)
y_sc = LinearScale()

bar = Bars(x=x_data, y=y_data, scales={'x': x_ord, 'y': y_sc}, type='stacked',
           colors=["#e41a1c", "#4daf4a"], labels=['Infecté', 'Sain'], padding=0.6,
           display_legend=True)
bar.label_display = True
bar.label_display_format = '.2%'

ax_x = Axis(scale=x_ord, grid_lines='solid')
ax_y = Axis(scale=y_sc, orientation='vertical', grid_lines='solid', tick_format='.0%',
            label='Risque d\'être infecté')

srange = FloatSlider(min=sensibility_low,
                     max=sensibility_high,
                     step=0.001,
                     description='sensibilité',
                     readout_format='.1%')
sprange = FloatSlider(min=specificity_low,
                      max=specificity_high,
                      value=specificity_high,
                      step=0.001,
                      description='Faux positifs',
                      readout_format='.1%')
crange = FloatSlider(min = contamination_low,
                     max = contamination_high,
                     step=0.001,
                     description='contamination',
                     readout_format='.1%')

def update_bar(contamination, selectivity, specificity):
    bar.y = compute_proba(contamination, selectivity, specificity)
    
srange.observe(lambda c: update_bar(crange.value, c['new'], sprange.value), names='value')
sprange.observe(lambda c: update_bar(crange.value, srange.value, c['new']), names='value')
crange.observe(lambda c: update_bar(c['new'], srange.value, sprange.value), names='value')

smintext = FloatText(value=srange.min*100, description='min')
smaxtext = FloatText(value=srange.max*100, description='max')
spmintext = FloatText(value=sprange.min*100, description='min')
spmaxtext = FloatText(value=sprange.max*100, description='max')
cmintext = FloatText(value=crange.min*100, description='min')
cmaxtext = FloatText(value=crange.max*100, description='max')

def update_slider_min(src, target):
    target.min = src / 100.
    
def update_slider_max(xrs, target):
    target.max = src / 100.
    
smintext.observe(lambda c: update_slider_min(c['new'], srange), names='value')
smaxtext.observe(lambda c: update_slider_max(c['new'], srange), names='value')
spmintext.observe(lambda c: update_slider_min(c['new'], sprange), names='value')
spmaxtext.observe(lambda c: update_slider_max(c['new'], sprange), names='value')
cmintext.observe(lambda c: update_slider_min(c['new'], crange), names='value')
cmaxtext.observe(lambda c: update_slider_max(c['new'], crange), names='value')

fig = Figure(marks=[bar], axes=[ax_x, ax_y], title='Efficacité des tests', legend_location='top-right',
            theme='gg')
to_display = VBox((fig,
        HBox((srange, smintext, smaxtext)),
        HBox((sprange, spmintext, spmaxtext)),
        HBox((crange, cmintext, cmaxtext))))
to_display